# **VidQuest**
###*This NoteBook will use a YouTube video link to create some multiple-choice questions depending on the content of the video.*

1.   Mariam Salman (19L-1239)
1.   Mirza Zain Ali Nasir (19L-2277)
1.   Faseeh Ullah Jafar (19L-2273)
2.   Bilal Sardar (19L-1232)










# **link to text**

In [13]:
pip install youtube-dl SpeechRecognition imageio==2.4.1 pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import os
from pathlib import Path
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import warnings
import speech_recognition as sr
warnings.filterwarnings("ignore")

In [15]:
def Process_audio(fileName):

  txtf=open("The_audio.txt","w+")
  myaudio=AudioSegment.from_wav(fileName)
  chunks_length_ms=8000
  chunks=make_chunks(myaudio,chunks_length_ms)
  for i, chunk in enumerate(chunks):
    chunkName='./chunked/'+fileName+"_{0}.wav".format(i)
    print("I am Exporting",chunkName)
    chunk.export(chunkName,format="wav")
    File=chunkName
    r= sr.Recognizer()
    with sr.AudioFile(File) as source:
      audio_listened=r.listen(source)

    try:
      rec=r.recognize_google(audio_listened)
      txtf.write(rec+".")

    except sr.UnknownValueError:
      print("I dont recognize your audio")
    except sr.RequestError as e:
      print("could not get result")

try:
  os.makedirs("chunked")
except:
  pass


In [16]:
def UrlToAudio(VideoUrl):
  url=VideoUrl
  os.system("youtube-dl -x --audio-format wav " + url)
  # load audio and pad/trim it to fit 30 seconds
  base_path = Path(r"")
  for wav_file_path in base_path.glob("*.wav"):
      Process_audio(str(wav_file_path))
      break

# **MCQs**

## **Libraries**

In [17]:
!pip install --quiet transformers
!pip install --quiet sentencepiece
!pip install --quiet textwrap3
!pip install --quiet nltk

     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 44.2 MB/s 
     |████████████████████████████████| 163 kB 86.0 MB/s 
     |████████████████████████████████| 1.3 MB 5.1 MB/s 


In [18]:
!pip install --quiet ipython-autotime
%load_ext autotime

     |████████████████████████████████| 1.6 MB 5.1 MB/s 
time: 411 µs (started: 2022-10-30 07:13:37 +00:00)


## **summraizer**

In [19]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_model = summary_model.to(device)


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

time: 38.1 s (started: 2022-10-30 07:13:37 +00:00)


In [20]:
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

time: 3.45 ms (started: 2022-10-30 07:14:15 +00:00)


In [21]:
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from textwrap3 import wrap

def postprocesstext (content):
  final=""
  for sent in sent_tokenize(content):
    sent = sent.capitalize()
    final = final +" "+sent
  return final


def summarizer(text,model,tokenizer):
  text = text.strip().replace("\n"," ")
  text = "summarize: "+text
  # print (text)
  max_len = 512
  encoding = tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=3,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  min_length = 75,
                                  max_length=300)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
  summary = dec[0]
  summary = postprocesstext(summary)
  summary= summary.strip()

  return summary



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


time: 2.69 s (started: 2022-10-30 07:14:15 +00:00)


[nltk_data] Downloading package wordnet to /root/nltk_data...


## **answer Keywords**

In [22]:
!pip install --quiet git+https://github.com/boudinfl/pke.git@dc4d5f21e0ffe64c4df93c46146d29d1c522476b
!pip install --quiet flashtext==2.7

     |████████████████████████████████| 235 kB 4.9 MB/s 
time: 30.5 s (started: 2022-10-30 07:14:18 +00:00)


In [23]:
pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-9xwlfn_l
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-9xwlfn_l
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=353e831317b87c563d19da811f138a565568f616630d62e7aa692b6db23e1082
  Stored in directory: /tmp/pip-ephem-wheel-cache-gn3fwnis/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
Successfully built pke
  Attempting uninstall: pke
    Found existing installation: pke 1.8.1
    Uninstalling pke-1.8.1:
      Successfully uninstalled pke-1.8.1
time: 13.3 s (started: 2022-10-30 07:14:48 +00:00)


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback


def get_nouns_multipartite(content):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'PROPN','NOUN'}
        #pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')

        extractor.load_document(input=content,language='en',
                        stoplist=stoplist,
                        normalization=None)
        
        extractor.candidate_selection(pos=pos)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=15)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


time: 16.6 s (started: 2022-10-30 07:15:02 +00:00)


In [25]:
from flashtext import KeywordProcessor


def get_keywords(originaltext,summarytext):
  keywords = get_nouns_multipartite(originaltext)
  print ("keywords unsummarized: ",keywords)
  keyword_processor = KeywordProcessor()
  for keyword in keywords:
    keyword_processor.add_keyword(keyword)

  keywords_found = keyword_processor.extract_keywords(summarytext)
  keywords_found = list(set(keywords_found))
  print ("keywords_found in summarized: ",keywords_found)

  important_keywords =[]
  for keyword in keywords:
    if keyword in keywords_found:
      important_keywords.append(keyword)

  return important_keywords[:4]




time: 20.7 ms (started: 2022-10-30 07:15:18 +00:00)


## **Question Generation**

In [26]:
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

time: 43.1 s (started: 2022-10-30 07:15:18 +00:00)


In [27]:
def get_question(context,answer,model,tokenizer):
  text = "context: {} answer: {}".format(context,answer)
  encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)
  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question




time: 3.01 ms (started: 2022-10-30 07:16:01 +00:00)


## **Distractors**

In [28]:
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity

def get_distractors_wordnet(word):
    distractors=[]
    try:
      syn = wn.synsets(word,'n')[0]
      
      word= word.lower()
      orig_word = word
      if len(word.split())>0:
          word = word.replace(" ","_")
      hypernym = syn.hypernyms()
      if len(hypernym) == 0: 
          return distractors
      for item in hypernym[0].hyponyms():
          name = item.lemmas()[0].name()
          #print ("name ",name, " word",orig_word)
          if name == orig_word:
              continue
          name = name.replace("_"," ")
          name = " ".join(w.capitalize() for w in name.split())
          if name is not None and name not in distractors:
              distractors.append(name)
    except:
      print ("Wordnet distractors not found")
    return distractors


time: 3.62 ms (started: 2022-10-30 07:16:01 +00:00)


time: 25.9 ms (started: 2022-10-30 07:16:01 +00:00)


### Distractors

In [29]:
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

time: 305 ms (started: 2022-10-30 07:16:01 +00:00)


In [30]:
from google.colab import drive
drive.mount('/content/gdrive')
glove_file = 'gdrive/My Drive/distractors/glove.6B.300d.txt'
tmp_file = 'gdrive/My Drive/distractors/word2vec-glove.6B.300d.txt'

Mounted at /content/gdrive
time: 20.1 s (started: 2022-10-30 07:16:02 +00:00)


In [31]:
import os

if not os.path.isfile(glove_file):
    print("Glove embeddings not found. Please download and place them in the following path: " + glove_file)

time: 220 ms (started: 2022-10-30 07:16:22 +00:00)


In [32]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

time: 1min 49s (started: 2022-10-30 07:16:22 +00:00)


In [33]:
model.most_similar(positive=['love'], topn=10)

[('loves', 0.6420263051986694),
 ('passion', 0.6261469125747681),
 ('loved', 0.595270574092865),
 ('romantic', 0.5921086668968201),
 ('lovers', 0.5904368162155151),
 ('lover', 0.5841912031173706),
 ('you', 0.5837852954864502),
 ('me', 0.5835772752761841),
 ('affection', 0.5818219184875488),
 ('always', 0.5701121687889099)]

time: 2.06 s (started: 2022-10-30 07:18:12 +00:00)


In [34]:
def generate_distractors(answer, count):
    answer = str.lower(answer)
    
    ##Extracting closest words for the answer. 
    try:
        closestWords = model.most_similar(positive=[answer], topn=count)
    except:
        #In case the word is not in the vocabulary, or other problem not loading embeddings
        return []

    #Return count many distractors
    distractors = list(map(lambda x: x[0], closestWords))[0:count]
    
    return distractors

time: 1.69 ms (started: 2022-10-30 07:18:14 +00:00)


In [35]:
generate_distractors('oxygen', 4)

['hydrogen', 'nitrogen', 'helium', 'nutrients']

time: 101 ms (started: 2022-10-30 07:18:14 +00:00)


# **Gardio app**

In [ ]:
pip install gradio

In [41]:
import gradio as gr

context1 = gr.inputs.Textbox(lines=10, placeholder="Enter link here...")
output = gr.outputs.HTML(  label="Question and Answers")
radiobutton = gr.inputs.Radio(["Wordnet", "Gensim"])

def generate_question(context1,radiobutton):
  UrlToAudio(context1)
  f = open("The_audio.txt", "r")
  context=f.read()
  summary_text = summarizer(context,summary_model,summary_tokenizer)
  for wrp in wrap(summary_text, 150):
    print (wrp)
  # np = getnounphrases(summary_text,sentence_transformer_model,3)
  np =  get_keywords(context,summary_text)
  print ("\n\nNoun phrases",np)
  output=""
  for answer in np:
    ques = get_question(summary_text,answer,question_model,question_tokenizer)
    if radiobutton=="Wordnet":
      distractors = get_distractors_wordnet(answer)
    else:
      distractors = generate_distractors(answer.capitalize(),3)
      print(distractors)
    # output= output + ques + "\n" + "Ans: "+answer.capitalize() + "\n\n"
    output ="\n"+ output + "<b style='color:blue;'>" + ques + "</b>"
    # output = output + "<br>"
    output ="\n"+ output + "<b style='color:green;'>" + "Ans: " +answer.capitalize()+  "</b>"
    if len(distractors)>0:
      for distractor in distractors[:4]:
        output = output + "<b style='color:brown;'>" + distractor+  "</b>\n"
    output = output + "<br>"

  summary ="Summary: "+ summary_text
  for answer in np:
    summary = summary.replace(answer,"<b>"+answer+"</b>")
    summary = summary.replace(answer.capitalize(),"<b>"+answer.capitalize()+"</b>")
  output = output + "<p>"+summary+"</p>"
  return output


iface = gr.Interface(
  fn=generate_question, 
  inputs=[context1,radiobutton],
  title="VidQuest", 
  outputs=output)
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d34a35958ee73ff7.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_0.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_1.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_2.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_3.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_4.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_5.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_6.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_7.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_8.wav
I am Exporting ./chunked/OpenAI Whisper is every Python developers dream-_xVTgdpokH4.wav_9.wav
I am Exporting ./chunked/OpenAI Whisper is every P

(<gradio.routes.App at 0x7f5814e9db10>,
 'http://127.0.0.1:7860/',
 'https://d34a35958ee73ff7.gradio.app')

time: 3min 33s (started: 2022-10-30 07:30:22 +00:00)
